In [ ]:
# Imports

import os
import sys
import uuid
import yaml
from tqdm import tqdm
import pandas as pd
import numpy as np

sys.path.append("../openWakeWord")
sys.path.append("../piper-sample-generator")
import openwakeword
from openwakeword.data import generate_adversarial_texts, augment_clips, mmap_batch_generator
from openwakeword.utils import compute_features_from_generator
from generate_samples import generate_samples

from openspeechtointent.model import CitrinetModel
# Load model (this will also download the model if it is not already present)
mdl = CitrinetModel()

import torch
print(torch.cuda.device_count()) # make sure to use GPU for much faster speech generation

from IPython.display import Audio
from pathlib import Path

# set logging to debug
import logging
logging.basicConfig(level=logging.DEBUG)

# Download Data (do this once)

In [ ]:
# Download piper models to generate synthetic training clips

!wget -O ../piper-sample-generator/models/en_US-libritts_r-medium.pt 'https://github.com/rhasspy/piper-sample-generator/releases/download/v2.0.0/en_US-libritts_r-medium.pt'


In [ ]:
# # Download openWakeWord models

import os
os.makedirs("../openWakeWord/openwakeword/resources/models", exist_ok=True)
!wget https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.onnx -O ../openWakeWord/openwakeword/resources/models/embedding_model.onnx
!wget https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/embedding_model.tflite -O ../openWakeWord/openwakeword/resources/models/embedding_model.tflite
!wget https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.onnx -O ../openWakeWord/openwakeword/resources/models/melspectrogram.onnx
!wget https://github.com/dscripka/openWakeWord/releases/download/v0.5.1/melspectrogram.tflite -O ../openWakeWord/openwakeword/resources/models/melspectrogram.tflite


# Load Config for the model

In [ ]:
# Load the YAML config file

with open('../wakeword_training_configs/angela.yml', 'r') as file:
    config = yaml.safe_load(file)


# Generate Positive clips for Training Using Piper TTS

In [ ]:
# Generate positive clips with target wakeword phrase

generate_samples(
    text=config["target_phrase"][0], max_samples=config["n_samples"],
    batch_size=50,
    noise_scales=[0.98], noise_scale_ws=[0.98], length_scales=[0.8, 1.0, 1.3],
    output_dir=os.path.join(config["output_dir"], config['model_name'], "positive_train3"), auto_reduce_batch_size=True,
    file_names=[uuid.uuid4().hex + ".wav" for i in range(config["n_samples"])]
)
torch.cuda.empty_cache()

In [ ]:
# Listen to the samples

generated_clips = [str(i) for i in Path(os.path.join(config["output_dir"], config['model_name'], "positive_train")).glob("*.wav")]

print(len(generated_clips))

Audio(generated_clips[25])

In [ ]:
# Validate the positive training clips with openSpeechToIntent model

all_scores = []
for i in tqdm(generated_clips):
    matched_intents, scores, durations = mdl.match_intents(i, config["target_phrase"], softmax_scores=False)
    all_scores.append(scores.max())

# Save the results to a csv file
pd.DataFrame({"file_path": generated_clips, "scores": all_scores}).to_csv("../data/positive_training_clips_quality_scores.csv", index=False)


In [ ]:
# Generate positive validation clips with target wakeword phrase

generate_samples(
    text=config["target_phrase"], max_samples=config["n_samples_val"],
    batch_size=50,
    noise_scales=[0.98], noise_scale_ws=[0.98], length_scales=[0.8, 1.0, 1.3],
    output_dir=os.path.join(config["output_dir"], config['model_name'], "positive_val"), auto_reduce_batch_size=True,
    file_names=[uuid.uuid4().hex + ".wav" for i in range(config["n_samples"])]
)
torch.cuda.empty_cache()


In [ ]:
# Validate the positive validation clips with openSpeechToIntent model

generated_clips = [str(i) for i in Path(os.path.join(config["output_dir"], config['model_name'], "positive_val")).glob("*.wav")]

all_scores = []
for i in tqdm(generated_clips):
    matched_intents, scores, durations = mdl.match_intents(i, config["target_phrase"], softmax_scores=False)
    all_scores.append(scores.max())

# Save the results to a csv file
pd.DataFrame({"file_path": generated_clips, "scores": all_scores}).to_csv("../data/positive_validation_clips_quality_scores.csv", index=False)

# Generate Negative Adversarial clips for training

In [ ]:
# Define custom adversarial phrases to include

adversarial_phrases = config["custom_negative_phrases"]

# Generate phrases that have similar english phonemes to the target phrase (note: this is non-deterministic)

adversarial_phrases.extend(generate_adversarial_texts(
    input_text=config["target_phrase"][0],  # just use "angela" for this step
    N=1000,
    include_partial_phrase=1.0,
    include_input_words=0.2
))

# Remove duplicates
adversarial_phrases = list(set(adversarial_phrases))

len(adversarial_phrases)

In [ ]:
# Generate adversarial samples for training

generate_samples(
    text=adversarial_phrases, max_samples=config["n_samples"],
    batch_size=50,
    noise_scales=[0.98], noise_scale_ws=[0.98], length_scales=[0.8, 1.0, 1.3],
    output_dir=os.path.join(config["output_dir"], config['model_name'], "negative_train"), auto_reduce_batch_size=True,
    file_names=[uuid.uuid4().hex + ".wav" for i in range(config["n_samples"])]
)
torch.cuda.empty_cache()


In [ ]:
# Listen to the samples

generated_clips = [str(i) for i in Path(os.path.join(config["output_dir"], config['model_name'], "negative_train")).glob("*.wav")]

print(len(generated_clips))

Audio(generated_clips[8])

In [ ]:
# Generate adversarial samples for validation

generate_samples(
    text=adversarial_phrases, max_samples=config["n_samples_val"],
    batch_size=50,
    noise_scales=[0.98], noise_scale_ws=[0.98], length_scales=[0.8, 1.0, 1.3],
    output_dir=os.path.join(config["output_dir"], config['model_name'], "negative_val"), auto_reduce_batch_size=True,
    file_names=[uuid.uuid4().hex + ".wav" for i in range(config["n_samples"])]
)
torch.cuda.empty_cache()


# Augment Positive and Negative Clips and Generate Features

In [ ]:
# Define paths

positive_train_output_dir = os.path.join(config["output_dir"], config["model_name"], "positive_train")
positive_val_output_dir = os.path.join(config["output_dir"], config["model_name"], "positive_val")
negative_train_output_dir = os.path.join(config["output_dir"], config["model_name"], "negative_train")
negative_val_output_dir = os.path.join(config["output_dir"], config["model_name"], "negative_val")
feature_save_dir = os.path.join(config["output_dir"], config["model_name"])

# Get paths for impulse response and background audio files
rir_paths = [i.path for j in config["rir_paths"] for i in os.scandir(j)]
background_paths = []
if len(config["background_paths_duplication_rate"]) != len(config["background_paths"]):
    config["background_paths_duplication_rate"] = [1]*len(config["background_paths"])
for background_path, duplication_rate in zip(config["background_paths"], config["background_paths_duplication_rate"]):
    background_paths.extend([i.path for i in os.scandir(background_path)]*duplication_rate)

In [ ]:
## Create generators to augment positive clips with background noise and other augmentations

## Positive training clips, keeping only clips with scores equal to or above the 20th percentile
scores = pd.read_csv("../data/positive_training_clips_quality_scores.csv")
positive_clips_train = scores.file_path.values
percentile_cutoff = np.percentile(scores['scores'], 20)
positive_clips_train = [i for i, j in zip(positive_clips_train, scores["scores"]) if j >= percentile_cutoff]*config["augmentation_rounds"]
positive_clips_train_generator = augment_clips(positive_clips_train, total_length=config["total_length"],
                                                batch_size=config["augmentation_batch_size"],
                                                background_clip_paths=background_paths,
                                                RIR_paths=rir_paths)

## Include XTTSv2 common voice clones
scores = pd.read_csv("../data/positive_training_clips_xttsv2_cv_clone_quality_scores.csv")
scores = scores[scores.scores >= -0.3]  # manully selected value by listening to the clips
xttsv2_cv_clone_clips = scores.file_path.tolist()*2
xttsv2_cv_clone_clips_generator = augment_clips(xttsv2_cv_clone_clips, total_length=config["total_length"],
                                                batch_size=config["augmentation_batch_size"]//4, # smaller batch size for more variety
                                                background_clip_paths=background_paths,
                                                RIR_paths=[]) # no RIR reverberation for this data as the XTTSv2 clone already captures some room audio effects

## Positive validation clips, keeping only clips with scores equal to or above the 20th percentile
positive_clips_val = [str(i) for i in Path(positive_val_output_dir).glob("*.wav")]
scores = pd.read_csv("../data/positive_validation_clips_quality_scores.csv")
percentile_cutoff = np.percentile(scores['scores'], 20)
positive_clips_val = [i for i, j in zip(positive_clips_val, scores["scores"]) if j >= percentile_cutoff]*config["augmentation_rounds"]

positive_clips_val_generator = augment_clips(positive_clips_val, total_length=config["total_length"],
                                                batch_size=config["augmentation_batch_size"],
                                                background_clip_paths=background_paths,
                                                RIR_paths=rir_paths)

## Negative training and validation clips
negative_clips_train = [str(i) for i in Path(negative_train_output_dir).glob("*.wav")]*config["augmentation_rounds"]
negative_clips_train_generator = augment_clips(negative_clips_train, total_length=config["total_length"],
                                                batch_size=config["augmentation_batch_size"],
                                                background_clip_paths=background_paths,
                                                RIR_paths=rir_paths)

negative_clips_val = [str(i) for i in Path(negative_val_output_dir).glob("*.wav")]*config["augmentation_rounds"]
negative_clips_val_generator = augment_clips(negative_clips_val, total_length=config["total_length"],
                                                batch_size=config["augmentation_batch_size"],
                                                background_clip_paths=background_paths,
                                                RIR_paths=rir_paths)


In [ ]:
# Augment the clips and compute the openWakeWord features using the encoder
n_cpus = os.cpu_count()//2  # don't use threads, only full cores

compute_features_from_generator(positive_clips_train_generator, n_total=len(positive_clips_train),
                                clip_duration=config["total_length"],
                                output_file=os.path.join(feature_save_dir, "positive_features_train.npy"),
                                device="gpu" if torch.cuda.is_available() else "cpu",
                                ncpu=n_cpus if not torch.cuda.is_available() else 1)

compute_features_from_generator(xttsv2_cv_clone_clips_generator, n_total=len(xttsv2_cv_clone_clips),
                                clip_duration=config["total_length"],
                                output_file=os.path.join(feature_save_dir, "positive_features_train_xttsv2_cv_clone.npy"),
                                device="gpu" if torch.cuda.is_available() else "cpu",
                                ncpu=n_cpus if not torch.cuda.is_available() else 1)

compute_features_from_generator(negative_clips_train_generator, n_total=len(negative_clips_train),
                                clip_duration=config["total_length"],
                                output_file=os.path.join(feature_save_dir, "negative_features_train.npy"),
                                device="gpu" if torch.cuda.is_available() else "cpu",
                                ncpu=n_cpus if not torch.cuda.is_available() else 1)

compute_features_from_generator(positive_clips_val_generator, n_total=len(positive_clips_val),
                                clip_duration=config["total_length"],
                                output_file=os.path.join(feature_save_dir, "positive_features_val.npy"),
                                device="gpu" if torch.cuda.is_available() else "cpu",
                                ncpu=n_cpus if not torch.cuda.is_available() else 1)

compute_features_from_generator(negative_clips_val_generator, n_total=len(negative_clips_val),
                                clip_duration=config["total_length"],
                                output_file=os.path.join(feature_save_dir, "negative_features_val.npy"),
                                device="gpu" if torch.cuda.is_available() else "cpu",
                                ncpu=n_cpus if not torch.cuda.is_available() else 1)